# 1. Set Up

In [ ]:
#install mtcnn
!pip install mtcnn

#mount Goolge Drive
from google.colab import drive
drive.mount('/content/gdrive')

#set file path
path = "/content/gdrive/My Drive/iss/project/"

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
#import library
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.layers import Lambda, Flatten, Dense
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras import layers
import cv2
import os
import numpy as np
from numpy import genfromtxt
import pandas as pd
import h5py
import matplotlib.pyplot as plt

import mtcnn
from mtcnn.mtcnn import MTCNN
from matplotlib import pyplot as plt
from tensorflow.keras.models import load_model
from scipy.spatial.distance import cosine
from PIL import Image
from tensorflow.keras import backend as K

if K.backend()=='tensorflow':
    keras.backend.set_image_data_format("channels_last")

%matplotlib inline
%load_ext autoreload
%autoreload 2

# 2. Data Processing

In [ ]:
# create the detector, using default weights
detector = MTCNN()
def extract_face(filename, required_size=(160, 160)):
    image = Image.open(filename)
    # convert to RGB, if needed
    image = image.convert('RGB')
    # convert to array
    pixels = np.asarray(image)
    # detect faces in the image
    results = detector.detect_faces(pixels)
    # extract the bounding box from the first face
    x1, y1, width, height = results[0]['box']
    # deal with negative pixel index
    x1, y1 = abs(x1), abs(y1)
    x2, y2 = x1 + width, y1 + height
    # extract the face
    face = pixels[y1:y2, x1:x2]
    # resize pixels to the model size
    image = Image.fromarray(face)
    image = image.resize(required_size)
    face_array = np.asarray(image)
    return face_array



# 3. Load Model

In [ ]:
#Define Triplet_loss
def triplet_loss(y_true, y_pred, alpha = 0.2):
    
    anchor, positive, negative = y_pred[0], y_pred[1], y_pred[2]
  
    # Step 1: Compute the (encoding) distance between the anchor and the positive
    pos_dist = tf.reduce_sum(tf.square(tf.subtract(anchor, positive)), axis=-1)
    # Step 2: Compute the (encoding) distance between the anchor and the negative
    neg_dist = tf.reduce_sum(tf.square(tf.subtract(anchor, negative)), axis=-1)
    # Step 3: subtract the two previous distances and add alpha.
    basic_loss = tf.add(tf.subtract(pos_dist, neg_dist), alpha)
    # Step 4: Take the maximum of basic_loss and 0.0. Sum over the training examples.
    loss = tf.reduce_sum(tf.maximum(basic_loss, 0.0))
    
    return loss

#load pre-trained FaceNet Model
facenet_model = load_model(path+'facenet_keras.h5', custom_objects={ 'loss': triplet_loss })

# 4. Get image encoding

In [ ]:
from sklearn.preprocessing import Normalizer
in_encoder = Normalizer('l2')

def get_embedding(model, face):
    # scale pixel values
    face = face.astype('float32')
    # standardization
    mean, std = face.mean(), face.std()
    face = (face - mean) / std
    print(face.shape)
    face = cv2.resize(face,(160,160))
    face = np.expand_dims(face, axis=0)
    print(face.shape)
    encode = model.predict(face)[0]
    print(encode)
    return encode

def who_is_it(image_path, database, model):
    
    #Compute the target "encoding" for the image
    face = extract_face(image_path)
    encoding = get_embedding(facenet_model,face)
    encoding = in_encoder.transform(np.expand_dims(encoding, axis=0))[0]
    
    # Initialize "min_dist" to a large value
    min_dist = float("inf")
    
    # Loop over the database dictionary's names and encodings.
    for (name, db_enc) in database.items():
        
        # Compute L2 distance between the target "encoding" and the current db_enc
        dist = cosine(db_enc,encoding)

        # If this distance is less than the min_dist, then set min_dist to dist, and identity to name
        if dist < 0.5 and dist < min_dist:
            min_dist = dist
            identity = name

    
    if min_dist > 0.5:
        print("Not in the database.")
    else:
        print ("it's " + str(identity) + ", the distance is " + str(min_dist))
        
    return min_dist, identity

# 5. Build image encoding library

In [ ]:
database = {}
ben_face = extract_face(path+"/data/5 Celebrity Faces Dataset/val/ben_afflek/httpabsolumentgratuitfreefrimagesbenaffleckjpg.jpg")
database["ben_afflek"] = get_embedding(facenet_model,ben_face)
#print(get_embedding(facenet_model,ben_face).shape)
a_face = extract_face(path+"/data/5 Celebrity Faces Dataset/val/austin/photo_2020-09-26_05-37-27 (4).jpg")
database["austin"] = get_embedding(facenet_model,a_face)
a_face = extract_face(path+"/data/5 Celebrity Faces Dataset/val/madonna/httpecximagesamazoncomimagesIfmaBKWLACULSRjpg.jpg")
database["madonna"] = get_embedding(facenet_model,a_face)

#test
who_is_it(path+"/data/5 Celebrity Faces Dataset/val/ben_afflek/httpbpblogspotcomedLMjVpRGkSWexgsXjkNIAAAAAAAADWgFFtAUqBlhAsjpg.jpg",database,facenet_model)
who_is_it(path+"/data/5 Celebrity Faces Dataset/val/austin/photo_2020-09-26_05-37-27.jpg",database,facenet_model)
who_is_it(path+"/data/5 Celebrity Faces Dataset/val/madonna/httpecximagesamazoncomimagesIfmaBKWLACULSRjpg.jpg",database,facenet_model)

(160, 160, 3)
(1, 160, 160, 3)
[-0.35715967  0.56805825 -0.02246208 -1.5599493  -0.22274993  0.31206363
  2.2126389  -1.0551347   0.11349715 -0.97813     1.1421908  -0.6536971
  0.15160137 -0.74272907  1.1953169  -0.62218195 -2.5768816   0.4588946
 -0.21730083 -0.24980476  1.5028306  -0.03511811  0.05881213 -0.9723351
  0.06294286 -0.41399258  0.96276104 -1.5717623  -0.7984349   1.2099433
  0.14957878  0.99214303  1.8099713   0.29607832 -0.6879203   0.42563197
 -1.5447482  -0.4632311  -0.62419754 -0.8786627   1.4744818  -0.7568308
 -1.5859734  -0.73545605 -0.45129827 -0.23810981  0.21692064 -0.8924271
 -0.43760607 -0.24870864 -0.6028922  -1.3944112   1.206109   -0.9149063
  0.512636   -0.8122218  -1.5299289   1.0629773   0.93077    -2.1749358
 -0.87885475 -0.7761552  -1.471464   -1.3767503   0.9563707  -2.4899604
 -0.2860678  -0.7537708   0.11845914 -0.45969534 -0.26307565 -0.4807784
 -0.3989519  -1.0307485  -2.9253402  -0.28871417  0.1633484   0.06209685
 -0.30971226  0.01262298  0.36

(0.0, 'madonna')

# 6. Output Model and Encoding DB

In [ ]:
import csv

with open('dict.csv', 'w') as csv_file:  
    writer = csv.writer(csv_file)
    for key, value in database.items():
       value = list(value)
       writer.writerow([key, value])
    #for key, value in database.items():
       #value = list(value)
       #value = np.fromstring(value.tostring())
       #re.sub("\s+", ",", value.strip())
       #value = "[" + value +"]"
       #print(value)
       #writer.writerow([key, value])